In [20]:
!pip install -U langchain==0.0.208 deeplake openai tiktoken pydantic duckduckgo-search

  Using cached pydantic-2.1.1-py3-none-any.whl (370 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 59.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 4.5.0
    Uninstalling lxml-4.5.0:
      Successfully uninstalled lxml-4.5.0


In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [5]:
os.environ["ACTIVELOOP_TOKEN"] = getpass.getpass("YOUR-ACTIVELOOP-TOKEN>")

YOUR-ACTIVELOOP-TOKEN> ········


### The LLMs

In [ ]:
from langchain.llms import OpenAI

In [ ]:
import langchain
langchain.verbose = False
llm = OpenAI(model="text-davinci-003", temperature=0.9)

In [ ]:
text = "Suggest a personalized workout routine for someone looking to improve jump height and prefers outdoor activities"
print(llm(text))

### The Chains

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="text-davinci-003", temperature=0.9)
prompt = PromptTemplate(input_variables=["product"], template="What is a good name for a company that makes {product}?")
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
print(chain.run("eco-friendly water bottles"))

### The Memory

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="text-davinci-003", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [ ]:
conversation.predict(input="Tell me about yourself.")

In [ ]:
conversation.predict(input="What can you do?")

In [ ]:
conversation.predict(input="How can you help me with data analysis")

In [ ]:
print(conversation)

### Deep Lake VectorStore

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [7]:

texts = [
     "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

my_activeloop_org_id= "ptah23"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


|

Dataset(path='hub://ptah23/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['6e912f1a-33ed-11ee-bfff-5221bcf0254a',
 '6e913096-33ed-11ee-bfff-5221bcf0254a']

In [8]:
retrieval_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

In [11]:
from langchain.agents import initialize_agent, Tool, AgentType
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    )
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [12]:
response = agent.run("When was Napoleon born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleon was born.
Action: Retrieval QA System
Action Input: When was Napoleon born?
Observation:  Napoleon was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon was born on 15 August 1769.

> Finished chain.
Napoleon was born on 15 August 1769.


In [13]:
# load the existing Deep Lake dataset and specify the embedding function

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

db.add_documents(docs)

Deep Lake Dataset in hub://ptah23/langchain_course_from_zero_to_hero already exists, loading from the storage


|

Dataset(path='hub://ptah23/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['75b512ba-33ee-11ee-bfff-5221bcf0254a',
 '75b51422-33ee-11ee-bfff-5221bcf0254a']

In [14]:
llm = OpenAI(model="text-davinci-003", temperature=0)

retrieval_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    )
]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [15]:
agent.run("When was Michael Jordan born?")



> Entering new  chain...
 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.


'Michael Jordan was born on 17 February 1963.'

### Agents in LangChain

In [36]:
from langchain.llms import OpenAI
from langchain.agents import AgentType, load_tools, initialize_agent, Tool
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = OpenAI(model="text-davinci-003", temperature=0)

In [37]:
search = DuckDuckGoSearchAPIWrapper()

In [38]:
tools = [
    Tool(
        name = "duckduckgo-search",
        func=search.run,
        description="Useful for when you need to search the web to answer questions about current events"
    )
]

In [39]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True, max_iterations=6)

In [40]:
agent("What's the latest news about the Mars rover?")



> Entering new  chain...
 I should search the web for the latest news
Action: duckduckgo-search
Action Input: "Mars rover news"
Observation: Shortly before the rover's 11th anniversary on the Red Planet, its team helped guide it up a steep, slippery slope to examine meteor craters. On Aug. 5, NASA's Curiosity rover will notch its 11th year on Mars by doing what it does best: studying the Red Planet's surface. The intrepid bot recently investigated a location nicknamed "Jau ... Mars Exploration Rovers. News. The Latest from Mars. View By News Type: The intrepid rotorcraft may head skyward again within the week. Martian maps are full of monikers recognizing places on Earth, explorers, and even cartoon characters. Get the Mars Newsletter. Subscribe to newsletter. Follow the Mission. NASA's Ingenuity Mars helicopter throws a shadow on the dusty surface on Aug. 2, 2023. Ingenuity is ready for its closeup, Mr. DeMille. NASA's surprisingly long-lived Mars helicopter is the star ... On Aug. 

{'input': "What's the latest news about the Mars rover?",
 'output': 'On Aug. 5, NASA\'s Curiosity rover will notch its 11th year on Mars by doing what it does best: studying the Red Planet\'s surface. The intrepid bot recently investigated a location nicknamed "Jau" that is pockmarked with dozens of impact craters. Scientists have rarely gotten a close-up view of so many Martian craters in one place.'}

### Tools in LangChain

In [41]:
from langchain.llms import OpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [42]:
llm = OpenAI(model="text-davinci-003", temperature=0)
prompt = PromptTemplate(input_variables=["query"], template="Write a summary of the following text: {query}")

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [43]:
search = DuckDuckGoSearchAPIWrapper()
tools = [
    Tool(name="Search", func=search.run, description="Useful for finding information abou recent events"),
    Tool(name="Summarizer", func=summarize_chain.run, description="Useful for summarizing texts")
]

In [44]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [45]:
response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
response['output']



> Entering new  chain...
 I should search for recent news about the Mars rover and then summarize the results.
Action: Search
Action Input: Latest news about the Mars rover
Observation: Shortly before the rover's 11th anniversary on the Red Planet, its team helped guide it up a steep, slippery slope to examine meteor craters. On Aug. 5, NASA's Curiosity rover will notch its 11th year on Mars by doing what it does best: studying the Red Planet's surface. The intrepid bot recently investigated a location nicknamed "Jau ... The supposed sighting was detailed on a blog, UFO Sightings Daily, written by Scott C Waring. In an image originally sent back by Nasa's Mars Curiosity Rover, Waring claimed to have spotted ... Earth planning date: Tuesday July 25, 2023 The rover engineers yet again did a fantastic job navigating Curiosity through this tricky terrain of fractured blocks and sand. Our parking spot is on the rim of one of the larger craters within a cluster of craters that we have been

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xpDe9qgdMkDxJBKlyn0Mfnhq on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xpDe9qgdMkDxJBKlyn0Mfnhq on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer.
Final Answer: NASA's Curiosity rover is celebrating its 11th anniversary on Mars by exploring a cluster of meteor craters. The rover's team has successfully navigated it up a steep, slippery slope to examine the craters. On August 5th, the rover will celebrate its 11th anniversary on Mars by continuing to study the planet's surface.

> Finished chain.


"NASA's Curiosity rover is celebrating its 11th anniversary on Mars by exploring a cluster of meteor craters. The rover's team has successfully navigated it up a steep, slippery slope to examine the craters. On August 5th, the rover will celebrate its 11th anniversary on Mars by continuing to study the planet's surface."